In [ ]:
def evaluate_model(dataset_val, inference_config, AP_threshold):
    print("\n\n")
    print("*"*30, " Evaluating model ", "*"*30)
    print("\n\n")

    # Compute VOC-Style mAP @ IoU=0.5
    # Running on 10 images. Increase for better accuracy.
    # image_ids = np.random.choice(dataset_val.image_ids, 10)

    image_ids = dataset_val.image_ids

    model = get_inference_model(inference_config)
    incorrect = []

    APs = []
    for i, image_id in enumerate(image_ids):
        print(f"Evaluating image {i}/{len(image_ids)}", end='\r')

        # Load image and ground truth data
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, inference_config,
                                   image_id, use_mini_mask=False)
        # molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=IOU_THRESHOLD)
        APs.append(AP)

        if AP < AP_threshold:
            incorrect.append(AP)



    print("Finished evaluating")
    return np.mean(APs), incorrect
